In [1]:
!pip install opendatasets
import opendatasets as od
import pandas
od.download("https://www.kaggle.com/datasets/msambare/fer2013")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key: MANIKEY
Dataset URL: https://www.kaggle.com/datasets/msambare/fer2013


100%|██████████| 60.3M/60.3M [00:00<00:00, 166MB/s]


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os

# Set paths for dataset
train_dir = "/content/fer2013/train"
val_dir = "/content/fer2013/test"

# Hyperparameters
IMG_SIZE = (224, 224)  # MobileNetV2 input size
BATCH_SIZE = 32
EPOCHS = 25
LEARNING_RATE = 0.0001
NUM_CLASSES = len(os.listdir(train_dir))  # Number of emotion categories

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

# Load pre-trained model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

# Freeze base layers
base_model.trainable = False

# Add custom layers for emotion recognition
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.5)(x)
output = Dense(NUM_CLASSES, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    verbose=1
)

# Save the final model
model.save("emotion_recognition_model.h5")

print("Training completed. Model saved as 'emotion_recognition_model.h5' and best model as 'best_emotion_model.h5'.")


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Epoch 1/25
897/897 ━━━━━━━━━━━━━━━━━━━━ 389s 427ms/step - accuracy: 0.2110 - loss: 1.8899 - val_accuracy: 0.2669 - val_loss: 1.7550
Epoch 2/25
  1/897 ━━━━━━━━━━━━━━━━━━━━ 1:56 130ms/step - accuracy: 0.2812 - loss: 1.8097

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


897/897 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.2812 - loss: 1.8097 - val_accuracy: 0.5000 - val_loss: 1.6381
Epoch 3/25
897/897 ━━━━━━━━━━━━━━━━━━━━ 379s 420ms/step - accuracy: 0.2779 - loss: 1.7583 - val_accuracy: 0.2963 - val_loss: 1.7246
Epoch 4/25
897/897 ━━━━━━━━━━━━━━━━━━━━ 0s 79us/step - accuracy: 0.3438 - loss: 1.6428 - val_accuracy: 0.3000 - val_loss: 1.7269
Epoch 5/25
897/897 ━━━━━━━━━━━━━━━━━━━━ 382s 418ms/step - accuracy: 0.2996 - loss: 1.7204 - val_accuracy: 0.3029 - val_loss: 1.7048
Epoch 6/25
897/897 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.3750 - loss: 1.5665 - val_accuracy: 0.4000 - val_loss: 1.6400
Epoch 7/25
897/897 ━━━━━━━━━━━━━━━━━━━━ 434s 422ms/step - accuracy: 0.3083 - loss: 1.7062 - val_accuracy: 0.3192 - val_loss: 1.6841
Epoch 8/25
897/897 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.3750 - loss: 1.5927 - val_accuracy: 0.2000 - val_loss: 1.6395
Epoch 9/25
897/897 ━━━━━━━━━━━━━━━━━━━━ 427s 418ms/step - accuracy: 0.3202 - loss: 1.6951 - val_ac

Training completed. Model saved as 'emotion_recognition_model.h5' and best model as 'best_emotion_model.h5'.


In [4]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from PIL import Image
import os

# Load the trained model
model_path = "/content/emotion_recognition_model.h5"  # Replace with the path to your model
model = load_model(model_path)

# Emotion classes
class_labels = ["angry😠", "disgust🤢", "fear😱", "happy☺️", "neutral😐", "sad😞", "surprise🎊"]

# Preprocess the image
def preprocess_image(image_path):
    try:
        img = Image.open(image_path).convert("RGB")
        img = img.resize((224, 224))  # Resize to the input size of the model
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
        img_array = img_array / 255.0  # Normalize pixel values
        return img_array
    except Exception as e:
        print(f"Error processing image: {e}")
        return None

# Test the model
def test_model(image_path):
    print(f"Testing with image: {image_path}")
    img_array = preprocess_image(image_path)
    if img_array is not None:
        predictions = model.predict(img_array)
        predicted_class = class_labels[np.argmax(predictions)]
        confidence = np.max(predictions)
        print(f"Predicted Emotion: {predicted_class}")
        print(f"Confidence: {confidence:.2f}")
    else:
        print("Image preprocessing failed.")

# Example usage
if __name__ == "__main__":
    # Path to a test image
    test_image_path = "/content/5.jpg"  # Replace with the path to your test image

    if os.path.exists(test_image_path):
        test_model(test_image_path)
    else:
        print("Test image not found. Please check the file path.")


Test image not found. Please check the file path.
